# GRAPE: Interactive Phylogenetic Analysis

This notebook provides an interactive introduction to GRAPE (Graph Analysis and Phylogenetic Estimation). You'll learn how to:

1. **Load and explore linguistic data**
2. **Run basic phylogenetic analysis**  
3. **Visualize phylogenetic trees**
4. **Compare different parameters**
5. **View all language family results**

## Prerequisites

Make sure you have the required dependencies installed:
```bash
pip install networkx ete3 numpy pandas matplotlib seaborn
```

In [1]:
import os
import sys
import subprocess
import pandas as pd
import numpy as np
from ete3 import Tree

# Change to the main GRAPE directory
os.chdir('../..')
print(f"Current directory: {os.getcwd()}")
print("✓ All dependencies loaded successfully!")

Current directory: /home/tiagot/tiatre/grape
✓ All dependencies loaded successfully!


## 1. Data Exploration

Let's start by exploring our linguistic datasets. GRAPE works with cognate data in TSV format.

In [2]:
# List available datasets
data_files = [f for f in os.listdir('data/') if f.endswith('.tsv')]
print(f"Available datasets: {data_files}")

# Load and examine the Dravidian dataset
dravlex_df = pd.read_csv('data/dravlex.tsv', sep='\t')

print(f"\nDravidian Dataset (dravlex.tsv):")
print(f"Shape: {dravlex_df.shape}")
print(f"Columns: {list(dravlex_df.columns)}")
print(f"\nFirst 5 rows:")
display(dravlex_df.head())

Available datasets: ['walworthpolynesian.tsv', 'tuled.tsv', 'chaconarawakan.tsv', 'iecor_german.tsv', 'iecor_small.tsv', 'dravlex.tsv', 'iecor_full.tsv', 'harald_ie.tsv']

Dravidian Dataset (dravlex.tsv):
Shape: (2114, 3)
Columns: ['Language', 'Parameter', 'Cognateset']

First 5 rows:


,Language,Parameter,Cognateset
0,Badga,I,i.001
1,Betta_Kurumba,I,i.001
2,Brahui,I,i.001
3,Gondi,I,i.001
4,Kannada,I,i.001


In [3]:
# Analyze the dataset structure
languages = dravlex_df['Language'].unique()
concepts = dravlex_df['Parameter'].unique()
cognate_sets = dravlex_df['Cognateset'].unique()

print(f"Number of languages: {len(languages)}")
print(f"Number of concepts: {len(concepts)}")
print(f"Number of cognate sets: {len(cognate_sets)}")

print(f"\nLanguages: {', '.join(sorted(languages))}")

# Check data completeness
data_matrix = dravlex_df.groupby(['Language', 'Parameter']).size().unstack(fill_value=0)
completeness = (data_matrix > 0).sum(axis=1) / len(concepts) * 100

print(f"\nData completeness by language:")
for lang in sorted(completeness.index):
    print(f"  {lang}: {completeness[lang]:.1f}%")

Number of languages: 20
Number of concepts: 100
Number of cognate sets: 778

Languages: Badga, Betta_Kurumba, Brahui, Gondi, Kannada, Kodava, Kolami, Kota, Koya, Kurukh, Kuwi, Malayalam, Malto, Ollari_Gadba, Parji, Tamil, Telugu, Toda, Tulu, Yeruva

Data completeness by language:
  Badga: 95.0%
  Betta_Kurumba: 100.0%
  Brahui: 100.0%
  Gondi: 100.0%
  Kannada: 100.0%
  Kodava: 100.0%
  Kolami: 96.0%
  Kota: 91.0%
  Koya: 100.0%
  Kurukh: 100.0%
  Kuwi: 56.0%
  Malayalam: 100.0%
  Malto: 95.0%
  Ollari_Gadba: 59.0%
  Parji: 64.0%
  Tamil: 100.0%
  Telugu: 100.0%
  Toda: 100.0%
  Tulu: 98.0%
  Yeruva: 99.0%


## 2. Basic GRAPE Analysis

Now let's run our first GRAPE analysis on the Dravidian data:

In [4]:
def run_grape(dataset, **kwargs):
    """Helper function to run GRAPE and return results."""
    cmd = ['python', 'grape.py', f'data/{dataset}', '--seed', '42']
    
    for key, value in kwargs.items():
        cmd.extend([f'--{key}', str(value)])
    
    print(f"Running: {' '.join(cmd)}")
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        return result.stderr  # GRAPE logs to stderr
    except subprocess.CalledProcessError as e:
        print(f"Error: {e.stderr}")
        return None

# Run basic analysis on Dravidian data
print("=== GRAPE Analysis: Dravidian Languages ===\n")
dravidian_output = run_grape('dravlex.tsv')
print(dravidian_output)

=== GRAPE Analysis: Dravidian Languages ===

Running: python grape.py data/dravlex.tsv --seed 42


[INFO] Detected CSV dialect: Delimiter='	', Quotechar='"', Lineterminator='\r\n'
[INFO] Graph built successfully.
[INFO] Phylogeny: 
         /-Badga
      /-|
     |  |   /-Toda
     |   \-|
     |     |   /-Betta_Kurumba
     |      \-|
   /-|         \-Kota
  |  |
  |  |   /-Tulu
  |  |  |
  |  |  |   /-Malayalam
  |   \-|--|
  |     |   \-Tamil
  |     |
  |     |   /-Kannada
--|      \-|
  |        |   /-Kodava
  |         \-|
  |            \-Yeruva
  |
  |      /-Brahui
  |   /-|
  |  |  |   /-Kurukh
  |  |   \-|
  |  |      \-Malto
   \-|
     |      /-Parji
     |   /-|
     |  |  |   /-Kuwi
     |  |   \-|
      \-|      \-Ollari_Gadba
        |
        |   /-Gondi
         \-|
           |   /-Kolami
            \-|
              |   /-Koya
               \-|
                  \-Telugu
[INFO] Newick format tree: (((Badga:0.1,(Toda:0.1,(Betta_Kurumba:0.1,Kota:0.1):0.1):0.1):0.3,(Tulu:0.1,(Malayalam:0.6,Tamil:0.6):0.1,(Kannada:0.1,(Kodava:0.8,Yeruva:0.8):0.1):0.1):0.3):0.9,((B

In [5]:
def extract_newick_tree(grape_output):
    """Extract Newick tree string from GRAPE output."""
    lines = grape_output.split('\n')
    for line in lines:
        if '[INFO] Newick format tree:' in line:
            return line.split('[INFO] Newick format tree: ', 1)[1].strip()
    return None

def visualize_tree(newick_string, title="Phylogenetic Tree"):
    """Create a visualization of the phylogenetic tree."""
    if not newick_string:
        print("No tree to visualize")
        return None
        
    tree = Tree(newick_string)
    
    print(f"\n{title}:")
    print("=" * len(title))
    print(tree.get_ascii(show_internal=True))
    
    # Show some tree statistics
    leaves = tree.get_leaves()
    print(f"\nTree Statistics:")
    print(f"  Number of languages: {len(leaves)}")
    print(f"  Tree height: {tree.get_farthest_leaf()[1]:.3f}")
    print(f"  Languages: {', '.join(sorted([leaf.name for leaf in leaves]))}")
    
    return tree

# Extract and visualize the Dravidian tree
dravidian_newick = extract_newick_tree(dravidian_output)
dravidian_tree = visualize_tree(dravidian_newick, "Dravidian Language Family Tree")


Dravidian Language Family Tree:

         /-Badga
      /-|
     |  |   /-Toda
     |   \-|
     |     |   /-Betta_Kurumba
     |      \-|
   /-|         \-Kota
  |  |
  |  |   /-Tulu
  |  |  |
  |  |  |   /-Malayalam
  |   \-|--|
  |     |   \-Tamil
  |     |
  |     |   /-Kannada
--|      \-|
  |        |   /-Kodava
  |         \-|
  |            \-Yeruva
  |
  |      /-Brahui
  |   /-|
  |  |  |   /-Kurukh
  |  |   \-|
  |  |      \-Malto
   \-|
     |      /-Parji
     |   /-|
     |  |  |   /-Kuwi
     |  |   \-|
      \-|      \-Ollari_Gadba
        |
        |   /-Gondi
         \-|
           |   /-Kolami
            \-|
              |   /-Koya
               \-|
                  \-Telugu

Tree Statistics:
  Number of languages: 20
  Tree height: 4.600
  Languages: Badga, Betta_Kurumba, Brahui, Gondi, Kannada, Kodava, Kolami, Kota, Koya, Kurukh, Kuwi, Malayalam, Malto, Ollari_Gadba, Parji, Tamil, Telugu, Toda, Tulu, Yeruva


## 3. Linguistic Validation

Let's check if our results align with known linguistic classifications:

In [6]:
def validate_grouping(tree, group_name, expected_languages):
    """Validate if expected languages form a monophyletic group."""
    if not tree:
        return False
        
    # Find languages that are actually in the tree
    tree_languages = {leaf.name for leaf in tree.get_leaves()}
    available_languages = expected_languages.intersection(tree_languages)
    
    if len(available_languages) < 2:
        print(f"\n{group_name}: Not enough languages in tree ({len(available_languages)})")
        return False
        
    # Get MRCA of the expected languages
    target_nodes = [leaf for leaf in tree.get_leaves() if leaf.name in available_languages]
    
    if len(target_nodes) < 2:
        return False
        
    mrca = tree.get_common_ancestor(target_nodes)
    clade_languages = {leaf.name for leaf in mrca.get_leaves()}
    
    is_monophyletic = available_languages.issubset(clade_languages)
    extra_languages = clade_languages - available_languages
    
    print(f"\n{group_name} Validation:")
    print(f"  Expected: {sorted(available_languages)}")
    print(f"  Found in clade: {sorted(clade_languages)}")
    print(f"  Monophyletic: {'✓' if is_monophyletic else '✗'}")
    
    if extra_languages:
        print(f"  Extra languages: {sorted(extra_languages)}")
    
    return is_monophyletic

# Define expected Dravidian groupings based on linguistic consensus
dravidian_groups = {
    'South Dravidian': {'Tamil', 'Malayalam', 'Kannada', 'Tulu', 'Kodava', 'Badga', 'Kota', 'Toda'},
    'Central Dravidian': {'Gondi', 'Koya', 'Kuwi', 'Kolami', 'Parji', 'Ollari_Gadba'},
    'North Dravidian': {'Brahui', 'Kurukh', 'Malto'}
}

# Validate groupings
print("=== Linguistic Validation ===")
validation_results = {}

for group_name, languages in dravidian_groups.items():
    validation_results[group_name] = validate_grouping(dravidian_tree, group_name, languages)

success_count = sum(validation_results.values())
total_count = len(validation_results)
print(f"\nOverall validation success: {success_count}/{total_count} groups ({success_count/total_count*100:.1f}%)")

=== Linguistic Validation ===

South Dravidian Validation:
  Expected: ['Badga', 'Kannada', 'Kodava', 'Kota', 'Malayalam', 'Tamil', 'Toda', 'Tulu']
  Found in clade: ['Badga', 'Betta_Kurumba', 'Kannada', 'Kodava', 'Kota', 'Malayalam', 'Tamil', 'Toda', 'Tulu', 'Yeruva']
  Monophyletic: ✓
  Extra languages: ['Betta_Kurumba', 'Yeruva']

Central Dravidian Validation:
  Expected: ['Gondi', 'Kolami', 'Koya', 'Kuwi', 'Ollari_Gadba', 'Parji']
  Found in clade: ['Gondi', 'Kolami', 'Koya', 'Kuwi', 'Ollari_Gadba', 'Parji', 'Telugu']
  Monophyletic: ✓
  Extra languages: ['Telugu']

North Dravidian Validation:
  Expected: ['Brahui', 'Kurukh', 'Malto']
  Found in clade: ['Brahui', 'Kurukh', 'Malto']
  Monophyletic: ✓

Overall validation success: 3/3 groups (100.0%)


## 4. Parameter Comparison

Let's explore how different parameters affect the results:

In [7]:
# Compare different community detection algorithms
algorithms = ['louvain', 'greedy']
algorithm_results = {}

print("=== Algorithm Comparison ===\n")

for alg in algorithms:
    print(f"\nTesting {alg} algorithm:")
    output = run_grape('dravlex.tsv', community=alg, strategy='fixed', initial_value=0.5)
    
    if output:
        newick = extract_newick_tree(output)
        if newick:
            tree = Tree(newick)
            algorithm_results[alg] = {
                'tree': tree,
                'newick': newick,
                'num_leaves': len(tree.get_leaves()),
                'tree_height': tree.get_farthest_leaf()[1]
            }
            
            print(f"  Tree height: {algorithm_results[alg]['tree_height']:.3f}")
            print(f"  Number of leaves: {algorithm_results[alg]['num_leaves']}")

# Compare tree topologies
if len(algorithm_results) >= 2:
    algs = list(algorithm_results.keys())
    tree1 = algorithm_results[algs[0]]['tree']
    tree2 = algorithm_results[algs[1]]['tree']
    
    # Calculate Robinson-Foulds distance
    rf_distance = tree1.robinson_foulds(tree2)
    print(f"\nRobinson-Foulds distance between {algs[0]} and {algs[1]}: {rf_distance[0]}")
    print(f"Normalized RF distance: {rf_distance[0]/rf_distance[1]:.3f}")

=== Algorithm Comparison ===


Testing louvain algorithm:
Running: python grape.py data/dravlex.tsv --seed 42 --community louvain --strategy fixed --initial_value 0.5


  Tree height: 4.600
  Number of leaves: 20

Testing greedy algorithm:
Running: python grape.py data/dravlex.tsv --seed 42 --community greedy --strategy fixed --initial_value 0.5


  Tree height: 4.600
  Number of leaves: 20

Robinson-Foulds distance between louvain and greedy: 0
Normalized RF distance: 0.000


## 5. All Language Family Trees

Let's display the phylogenetic trees for all the language families that GRAPE has analyzed:

In [ ]:
# Display trees for all analyzed language families
def display_all_trees():
    """Display ASCII trees and publication image links for all language families."""
    tree_files = {
        'Dravidian': 'docs/images/trees/dravidian_formatted.txt',
        'Polynesian': 'docs/images/trees/polynesian_formatted.txt', 
        'Indo-European': 'docs/images/trees/indo-european_formatted.txt',
        'Tupian': 'docs/images/trees/tupian_formatted.txt',
        'Arawakan': 'docs/images/trees/arawakan_formatted.txt'
    }
    
    publication_images = {
        'Romance': 'docs/images/trees/publication/romance.png',
        'Austroasiatic': 'docs/images/trees/publication/austroasiatic.png',
        'Turkic': 'docs/images/trees/publication/turkic.png',
        'Dravidian': 'docs/images/trees/publication/dravidian.png',
        'Polynesian': 'docs/images/trees/publication/polynesian.png',
        'Tupian': 'docs/images/trees/publication/tupian.png'
    }
    
    print("🌳 GRAPE PHYLOGENETIC TREE VISUALIZATIONS")
    print("=" * 50)
    
    print("\n📊 PUBLICATION-QUALITY IMAGES:")
    print("High-resolution images suitable for academic papers and presentations:\n")
    
    for family, image_path in publication_images.items():
        if os.path.exists(image_path):
            print(f"✅ {family} Language Family:")
            print(f"   📊 Publication PNG: {image_path}")
            print(f"   📊 Publication SVG: {image_path.replace('.png', '.svg')}")
        else:
            print(f"⚠️  {family}: Publication images not yet generated")
    
    print("\n📝 ASCII TREE REPRESENTATIONS:")
    print("Text-based tree visualizations with linguistic analysis:\n")
    
    for family, filepath in tree_files.items():
        if os.path.exists(filepath):
            print(f"✅ {family} Language Family: {filepath}")
            
            # Display first few lines of the tree
            with open(filepath, 'r') as f:
                lines = f.readlines()[:15]  # First 15 lines
                print("   Preview:")
                for line in lines:
                    print(f"   {line.rstrip()}")
                print("   [... truncated for brevity ...]\n")
        else:
            print(f"❌ Tree file not found for {family}: {filepath}")
    
    print("\n🎯 USAGE IN PUBLICATIONS:")
    print("LaTeX: \\includegraphics[width=0.8\\textwidth]{docs/images/trees/publication/romance.png}")
    print("Markdown: ![Romance Tree](docs/images/trees/publication/romance.png)")
    print("HTML: <img src='docs/images/trees/publication/romance.png' alt='Romance Tree' width='800'>")

# Display all available trees and publication images
display_all_trees()

## 6. Summary

Let's summarize our key findings:

In [9]:
print("=" * 60)
print("GRAPE ANALYSIS SUMMARY")
print("=" * 60)

print("\n📊 DATA EXPLORATION:")
print(f"• Analyzed {len(languages)} Dravidian languages across {len(concepts)} concepts")
print(f"• Data completeness ranges from {completeness.min():.1f}% to {completeness.max():.1f}%")
print(f"• Dataset contains {len(cognate_sets)} unique cognate sets")

print("\n🌳 PHYLOGENETIC RESULTS:")
if 'validation_results' in locals():
    success_rate = sum(validation_results.values()) / len(validation_results) * 100
    print(f"• Linguistic validation success rate: {success_rate:.1f}%")
    for group, success in validation_results.items():
        status = "✓" if success else "✗"
        print(f"  {status} {group}")

print("\n⚙️ PARAMETER ANALYSIS:")
if 'algorithm_results' in locals():
    print(f"• Tested {len(algorithm_results)} community detection algorithms")
    if len(algorithm_results) >= 2:
        print(f"• Robinson-Foulds distance between algorithms: {rf_distance[0]}")

print("\n🔍 KEY INSIGHTS:")
print("• GRAPE successfully recovers established linguistic groupings")
print("• Community detection algorithms show consistent results")
print("• Tree visualizations provide clear phylogenetic relationships")
print("• Random seed (--seed 42) ensures reproducible results")

print("\n📋 RECOMMENDATIONS:")
print("• Use Louvain algorithm for most analyses (faster)")
print("• Use Greedy algorithm when reproducibility is critical")
print("• Always use --seed parameter for reproducible results")
print("• Validate results against known linguistic classifications")

print("\n" + "=" * 60)
print("🎉 Analysis complete! GRAPE provides a powerful framework for")
print("   phylogenetic inference that complements traditional methods.")
print("=" * 60)

GRAPE ANALYSIS SUMMARY

📊 DATA EXPLORATION:
• Analyzed 3 Dravidian languages across 100 concepts
• Data completeness ranges from 56.0% to 100.0%
• Dataset contains 778 unique cognate sets

🌳 PHYLOGENETIC RESULTS:
• Linguistic validation success rate: 100.0%
  ✓ South Dravidian
  ✓ Central Dravidian
  ✓ North Dravidian

⚙️ PARAMETER ANALYSIS:
• Tested 2 community detection algorithms
• Robinson-Foulds distance between algorithms: 0

🔍 KEY INSIGHTS:
• GRAPE successfully recovers established linguistic groupings
• Community detection algorithms show consistent results
• Tree visualizations provide clear phylogenetic relationships
• Random seed (--seed 42) ensures reproducible results

📋 RECOMMENDATIONS:
• Use Louvain algorithm for most analyses (faster)
• Use Greedy algorithm when reproducibility is critical
• Always use --seed parameter for reproducible results
• Validate results against known linguistic classifications

🎉 Analysis complete! GRAPE provides a powerful framework for
   phy

## Next Steps

Now that you've completed this interactive analysis, you can:

1. **Explore your own data**: Replace the datasets with your own cognate data
2. **Try different parameters**: Experiment with various settings to optimize for your data
3. **Compare with traditional methods**: Use tools like BEAST, MrBayes, or IQ-TREE for comparison
4. **Validate results**: Check your results against published linguistic classifications
5. **Scale up**: Apply GRAPE to larger datasets for comprehensive analyses

## Documentation Links

For more information, see:
- [GRAPE Documentation](../README.md)
- [Parameter Guide](../user_guide/parameters.md)
- [Dravidian Walkthrough](dravidian_walkthrough.md)
- [Mathematical Background](../technical/mathematical_background.md)
- [Tree Visualizations](../images/trees/VISUALIZATION_SUMMARY.md)